In [ ]:
HTML(read(open("style.css"), String))

# Alpha-Beta Pruning


This notebook implements an AI which calculates the best move for a chess game using alpha-beta-Pruning algorithm.

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random

# Pkg.add("NBInclude")
using NBInclude

In [ ]:
@nbinclude("EvaluatePosition.ipynb")

In [ ]:
@nbinclude("Memoization.ipynb")

## AlphaBetaMax

The `alphaBetaMax_noMem` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does not use Memoization.

In [ ]:
function alphaBetaMax_noMem(State::Board, score::Int64, depth::Int64, alpha::Int64=-200000, beta::Int64=200000)::Int64
    if isterminal(State)
        return terminal_evaluation(State) - depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = alphaBetaMin_noMem(State, nextEval, depth - 1, alpha, beta)
        undomove!(State, undoinfo)
        if value >= beta
            return value
        end
        alpha = max(alpha, value)
    end
    return alpha
end

## AlphaBetaMin

The `alphaBetaMin_noMem` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the minimal centipawn evaluation of the current position for the player playing black where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does not use Memoization.

In [ ]:
function alphaBetaMin_noMem(State::Board, score::Int64, depth::Int64, alpha::Int64=-200000, beta::Int64=200000)::Int64
    if isterminal(State)
        return terminal_evaluation(State) + depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = alphaBetaMax_noMem(State, nextEval, depth - 1, alpha, beta)
        undomove!(State, undoinfo)
        if value <= alpha
            return value
        end
        beta = min(beta, value)
    end
    return beta
end

## Alpha-Beta-Pruning function

The `alphaBetaPruning_noMem` function takes in 3 arguments
1. `State` is the current state of type `Board`
1. `score` is the static centipawn evaluation of the static position
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function returns the best value and the best move the moving player can play in the current position. It calls the alpha-beta-pruning algorithm. If multiple moves are found which result in the best evaluation a random move will be chosen. This function does not use Memoization.

In [ ]:
function alphaBetaPruning_noMem(State::Board, score::Int64, depth::Int64)::Tuple{Int64, Move}
    next_moves = moves(State)
    BestMoves = []
    if sidetomove(State) == WHITE
        bestVal = alphaBetaMax_noMem(State, Int(score), Int(depth))
        for move in next_moves 
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move) 
            if alphaBetaMin_noMem(State, nextEval, depth - 1) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    elseif sidetomove(State) == BLACK
        bestVal = alphaBetaMin_noMem(State, score, depth)
        for move in next_moves 
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            if alphaBetaMax_noMem(State, nextEval, depth - 1) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

## Alpha-beta-Pruning with Memoization

In [ ]:
@nbinclude("Memoization.ipynb")

### AlphaBetaMax function with Memoization

The `alphaBetaMax` function takes in 4 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `cache` is a dictionary which stores the calculated values
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does use Memoization meaning it saves and uses calculated values stored the `Cache`.

In [ ]:
function alphaBetaMax(State::Board, score::Int64, hash::UInt64, depth::Int64, 
                      cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64=-200000, beta::Int64=200000)::Int64
    if isterminal(State)
        return terminal_evaluation(State) - depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval, nextHash = updateBoardData(State, score, hash, move)
        undoinfo = domove!(State, move)
        value = evaluate(State, alphaBetaMin, nextEval, nextHash, depth - 1, cache, alpha, beta)
        undomove!(State, undoinfo)
        if value >= beta
            return value
        end
        alpha = max(alpha, value)
    end
    return alpha
end

### AlphaBetaMin function with memoization

The Alpha-Beta-Min function takes in 4 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `cache` is a dictionary which stores the calculated values
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the minimal centipawn evaluation of the current position for the player playing black where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does use Memoization meaning it saves and uses calculated values stored the `Cache`.

In [ ]:
function alphaBetaMin(State::Board, score::Int64, hash::UInt64, depth::Int64, 
                      cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64=-200000, beta::Int64=200000)::Int64
    if isterminal(State)
        return terminal_evaluation(State) + depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval, nextHash = updateBoardData(State, score, hash, move)
        undoinfo = domove!(State, move)
        value = evaluate(State, alphaBetaMax, nextEval, nextHash, depth - 1, cache ,alpha, beta)
        undomove!(State, undoinfo)
        if value <= alpha
            return value
        end
        beta = min(beta, value)
    end
    return beta
end

The `alphaBetaPruning` function takes in 3 arguments and 1 optional argument.
1. `State` is the current state of type `Board`
1. `score` is the static centipawn evaluation of the static position
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `cache` is optional and is default empty dictionary. Cache is a dictionary which stores the calculated values

The function returns the best value and the best move the moving player can play in the current position. It calls the alpha-beta-pruning algorithm. If multiple moves are found which result in the best evaluation a random move will be chosen.

In [ ]:
function alphaBetaPruning(State::Board, score::Int64, hash::UInt64, depth::Int64,
                          cache::Dict{UInt64, Tuple{String, Int64, Int64}} = initCache())::Tuple{Int64, Move}
    next_moves = moves(State)
    BestMoves = []
    if sidetomove(State) == WHITE
        bestVal = evaluate(State, alphaBetaMax, score, hash, depth, cache)
        for move in next_moves
            nextEval, nextHash = updateBoardData(State, score, hash, move)
            undoinfo = domove!(State, move)
            if evaluate(State, alphaBetaMin, nextEval, nextHash, depth-1, cache) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    elseif sidetomove(State) == BLACK
        bestVal = evaluate(State, alphaBetaMin, score, hash, depth, cache)
        for move in next_moves 
            nextEval, nextHash = updateBoardData(State, score, hash, move)
            undoinfo = domove!(State, move)
            if evaluate(State, alphaBetaMax, nextEval, nextHash, depth-1, cache) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end